### Apply Assumption with Obligor By Field

In [1]:
from absbox import API,EnginePath

localAPI = API(EnginePath.DEV,check=False,lang='english')

Connecting engine server -> https://absbox.org/api/dev

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅Connected, local lib:0.46.5, server:0.50.1

#### Assets with obligor info

Let's setup asset with different tags/ids, just plug a map call `obligor` to the asset

    * Asset 1
    
      * ID: "1"
      * Tag:["A","B"]
      * Field: {"age":30,"creditLevel":"A"}

    * Asset 2
    
      * ID: "2"
      * Tag:["C","B"]
      * Field: {"age":50,"creditLevel":"B"}

In [2]:
ob1 = {
    "id":"1",
    "tag":["A","B"],
    "fields":{"age":30,"creditLevel":"A"}
} 

ob2 = {
    "id":"2",
    "tag":["C","B"],
    "fields":{"age":50,"creditLevel":"B"}
} 


myAsset1 = ["Mortgage"
            ,{"originBalance": 12000.0
             ,"originRate": ["fix",0.045]
             ,"originTerm": 120
             ,"freq": "monthly"
             ,"type": "level"
             ,"originDate": "2021-02-01"
             ,"obligor":ob1
             }
            ,{"currentBalance": 2000.0
             ,"currentRate": 0.075
             ,"remainTerm": 80
             ,"status": "current"}]
myAsset2 = ["Mortgage"
            ,{"originBalance": 12000.0
             ,"originRate": ["fix",0.045]
             ,"originTerm": 120
             ,"freq": "monthly"
             ,"type": "level"
             ,"originDate": "2021-02-01"
             ,"obligor":ob2
             }
            ,{"currentBalance": 5000.0
             ,"currentRate": 0.075
             ,"remainTerm": 80
             ,"status": "current"}]

myPool = {'assets':[myAsset1,myAsset2],
          'cutoffDate':"2022-03-01"}

ppyAssump = (("Mortgage",None ,{"CPR":0.1}, None, None)
             ,None
             ,None)
defAssump = (("Mortgage",{"CDR":0.2} ,None, None, None)
             ,None
             ,None)

#### assumption with obligor fields

User can set multiple conditions to apply asset assumptions to the ones match

In [4]:
AssetLevelAssumption = ("ByObligor"
                        ,("ByField",[("creditLevel","in",["A"]),("age","cmp","L",35)],ppyAssump)
                        ,("ByDefault",defAssump)
                       )

r = localAPI.runPool(myPool
                   ,poolAssump=AssetLevelAssumption
                   ,read=True)

r['PoolConsol']['flow'].head()

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-06-01,7000.00,0.00,0.00,0.00,0.00,0,0.00,0.075,None,None,0.00,0.00,0,0.00,0,0.00
2024-07-01,6825.24,66.66,43.07,17.24,90.86,0,90.86,0.075,None,None,66.66,17.24,0,90.86,0,90.86
2024-08-01,6650.48,66.00,41.97,17.49,91.27,0,91.27,0.075,None,None,132.66,34.73,0,182.13,0,182.13
2024-09-01,6479.28,65.37,40.89,17.16,88.67,0,88.67,0.075,None,None,198.03,51.89,0,270.80,0,270.80
2024-10-01,6314.83,64.77,39.86,16.30,83.38,0,83.38,0.075,None,None,262.80,68.19,0,354.18,0,354.18


Use mulitple `ByField` are allowed

In [5]:
AssetLevelAssumption = ("ByObligor"
                        ,("ByField",[("creditLevel","in",["A"]),("age","cmp","L",35)],ppyAssump)
                        ,("ByField",[("age","cmp","G",35)],defAssump)
                       )

r = localAPI.runPool(myPool
                   ,poolAssump=AssetLevelAssumption
                   ,read=True)

r['PoolConsol']['flow'].head()

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-06-01,7000.00,0.00,0.00,0.00,0.00,0,0.00,0.075,None,None,0.00,0.00,0,0.00,0,0.00
2024-07-01,6825.24,66.66,43.07,17.24,90.86,0,90.86,0.075,None,None,66.66,17.24,0,90.86,0,90.86
2024-08-01,6650.48,66.00,41.97,17.49,91.27,0,91.27,0.075,None,None,132.66,34.73,0,182.13,0,182.13
2024-09-01,6479.28,65.37,40.89,17.16,88.67,0,88.67,0.075,None,None,198.03,51.89,0,270.80,0,270.80
2024-10-01,6314.83,64.77,39.86,16.30,83.38,0,83.38,0.075,None,None,262.80,68.19,0,354.18,0,354.18


User can specify a `Range` for a user field

In [6]:
AssetLevelAssumption = ("ByObligor"
                        ,("ByField",[("age","range","II",25,40)],ppyAssump)
                       )

r = localAPI.runPool(myPool
                   ,poolAssump=AssetLevelAssumption
                   ,read=True)

r['PoolConsol']['flow'].head()

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-06-01,7000.00,0.00,0.00,0.00,0,0,0,0.075,None,None,0.00,0.00,0,0,0,0
2024-07-01,6915.22,67.54,43.64,17.24,0,0,0,0.075,None,None,67.54,17.24,0,0,0,0
2024-08-01,6829.94,67.79,43.10,17.49,0,0,0,0.075,None,None,135.33,34.73,0,0,0,0
2024-09-01,6744.73,68.05,42.57,17.16,0,0,0,0.075,None,None,203.38,51.89,0,0,0,0
2024-10-01,6660.12,68.31,42.04,16.30,0,0,0,0.075,None,None,271.69,68.19,0,0,0,0


User can setup a `Not` to negate the condition

In [7]:
AssetLevelAssumption = ("ByObligor"
                        ,("ByField",[("not",("age","range","II",25,60))],ppyAssump)
                       )

r = localAPI.runPool(myPool
                   ,poolAssump=AssetLevelAssumption
                   ,read=True)

r['PoolConsol']['flow'].head()

/home/xiaoyu/repo/AbsBox/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'absbox.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty,CumPrincipal,CumPrepay,CumDelinq,CumDefault,CumRecovery,CumLoss
Date,,,,,,,,,,,,,,,,
2024-06-01,7000.00,0.00,0.00,0,0,0,0,0.075,None,None,0.00,0,0,0,0,0
2024-07-01,6932.30,67.70,43.75,0,0,0,0,0.075,None,None,67.70,0,0,0,0,0
2024-08-01,6864.16,68.14,43.31,0,0,0,0,0.075,None,None,135.84,0,0,0,0,0
2024-09-01,6795.60,68.56,42.89,0,0,0,0,0.075,None,None,204.40,0,0,0,0,0
2024-10-01,6726.61,68.99,42.46,0,0,0,0,0.075,None,None,273.39,0,0,0,0,0
